In [34]:
import tkinter as tk

# Examples

## Temperature Converter

In [35]:
def fahrenheit_to_celsius():
    """Convert the value for Fahrenheit to Celsius and insert the
    result into lbl_result.
    """
    fahrenheit = ent_temperature.get()
    celsius = (5 / 9) * (float(fahrenheit) - 32)
    lbl_result["text"] = f"{round(celsius, 2)} \N{DEGREE CELSIUS}"

# Set up the window
window = tk.Tk()
window.title("Temperature Converter")
window.resizable(width=False, height=False)

# Create the Fahrenheit entry frame with an Entry
# widget and label in it
frm_entry = tk.Frame(master=window)
ent_temperature = tk.Entry(master=frm_entry, width=10)
lbl_temp = tk.Label(master=frm_entry, text="\N{DEGREE FAHRENHEIT}")

# Layout the temperature Entry and Label in frm_entry
# using the .grid() geometry manager
ent_temperature.grid(row=0, column=0, sticky="e")
lbl_temp.grid(row=0, column=1, sticky="w")

# Create the conversion Button and result display Label
btn_convert = tk.Button(
    master=window,
    text="\N{RIGHTWARDS BLACK ARROW}",
    command=fahrenheit_to_celsius
)
lbl_result = tk.Label(master=window, text="\N{DEGREE CELSIUS}")

# Set up the layout using the .grid() geometry manager
frm_entry.grid(row=0, column=0, padx=10)
btn_convert.grid(row=0, column=1, pady=10)
lbl_result.grid(row=0, column=2, padx=10)

# Run the application
window.mainloop()

## Text Editor

In [36]:
from tkinter.filedialog import askopenfilename, asksaveasfilename

def open_file():
    """Open a file for editing."""
    filepath = askopenfilename(
        filetypes=[("Text Files", "*.txt"), ("All Files", "*.*")]
    )
    if not filepath:
        return
    txt_edit.delete("1.0", tk.END)
    with open(filepath, mode="r", encoding="utf-8") as input_file:
        text = input_file.read()
        txt_edit.insert(tk.END, text)
    window.title(f"Simple Text Editor - {filepath}")

def save_file():
    """Save the current file as a new file."""
    filepath = asksaveasfilename(
        defaultextension=".txt",
        filetypes=[("Text Files", "*.txt"), ("All Files", "*.*")],
    )
    if not filepath:
        return
    with open(filepath, mode="w", encoding="utf-8") as output_file:
        text = txt_edit.get("1.0", tk.END)
        output_file.write(text)
    window.title(f"Simple Text Editor - {filepath}")

window = tk.Tk()
window.title("Simple Text Editor")

window.rowconfigure(0, minsize=800, weight=1)
window.columnconfigure(1, minsize=800, weight=1)

txt_edit = tk.Text(window)
frm_buttons = tk.Frame(window, relief=tk.RAISED, bd=2)
btn_open = tk.Button(frm_buttons, text="Open", command=open_file)
btn_save = tk.Button(frm_buttons, text="Save As...", command=save_file)

btn_open.grid(row=0, column=0, sticky="ew", padx=5, pady=5)
btn_save.grid(row=1, column=0, sticky="ew", padx=5)

frm_buttons.grid(row=0, column=0, sticky="ns")
txt_edit.grid(row=0, column=1, sticky="nsew")

window.mainloop()

# Backend Code Scratch

In [37]:
import wikipedia

search = "worthington ohio schools"

search_results = wikipedia.search(query=search, results=5)

# Print the top 5 results (titles)
print(f'Top 5 Wikipedia Results for "{search}":')
for result in search_results:
  print(result)

Top 5 Wikipedia Results for "worthington ohio schools":
Worthington, Ohio
Thomas Worthington High School
Worthington Christian High School
Worthington Kilbourne High School
Worthington City School District


# DuckDuckGo and Reddit

- How do I sort through all irrelevant comments such as "what is your budget?" and "thank you for your review"
- What is the optimal number of reddit posts to go through? What will the API let me do per minute and per day?
- How can I use NLP to fine tune the results?
- How can I use a LLM to fine tune the results or synthesize them?
- How to intuitively distinguish between posts and comment trees in resulting dataset
- Get the price

In [86]:
from duckduckgo_search import DDGS
import praw
import json
import pandas as pd
# get secrets
with open("../secrets.json") as f:
    secrets = json.load(f)
# initialize reddit class
reddit = praw.Reddit(
    client_id=secrets.get("client_id"),
    client_secret=secrets.get("client_secret"),
    user_agent=secrets.get("user_agent"),
)

In [109]:
search_text = "best leggings brand site:reddit.com"

In [110]:
results = DDGS().text(search_text, max_results=10)
all_comments_lists = []
for result in results:
    reddit_url = result["href"]
    submission = reddit.submission(url=reddit_url)
    submission.comments.replace_more(limit=1) # limit represents the comment tree, put None for getting the whole tree
    comments_list = [
        [reddit_url, comment.score, comment.body] for comment in submission.comments.list()
    ]
    all_comments_lists.extend(comments_list)

comments_df = pd.DataFrame(all_comments_lists, columns=['url', 'upvotes', 'comment'])
comments_df = comments_df.sort_values(by=['upvotes'], ascending=False)
comments_df

,url,upvotes,comment
706,https://www.reddit.com/r/femalefashionadvice/c...,411,This is amazing! Thank you for putting it toge...
2,https://www.reddit.com/r/femalefashionadvice/c...,343,My go to leggings are 90 degrees by reflex. De...
0,https://www.reddit.com/r/femalefashionadvice/c...,318,1. Costco has a brand of leggings called Matty...
3,https://www.reddit.com/r/femalefashionadvice/c...,277,Aerie leggings! Their PLAY leggings are thin b...
1,https://www.reddit.com/r/femalefashionadvice/c...,219,The Zella Live-In leggings from Nordstrom! The...
...,...,...,...
1166,https://www.reddit.com/r/minimalism/comments/g...,-2,I'm very similar to you in that I like to have...
477,https://www.reddit.com/r/gymsnark/comments/zun...,-2,It looks like you are posting about looking fo...
522,https://www.reddit.com/r/gymsnark/comments/zun...,-2,I second this. I ordered a few of her sports b...
858,https://www.reddit.com/r/femalefashionadvice/c...,-6,Polyester is made out of fossil fuels and I ca...


c:\Users\Max\.virtualenvs\experiments-0QODYWh7\Lib\site-packages\curl_cffi\aio.py:205: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


In [111]:

comments_df.to_csv("comments_data.csv", index=False)